In [2]:
import pandas as pd
from tqdm import tqdm

In [3]:
test_sum=pd.read_csv("test_summ.csv")
test_sum

,Unnamed: 0,summid,newsid,rewritten_title,userid
0,0,SUMM1,N24110,Legal battle looms over Trump EPA's rule chang...,NT1
1,1,SUMM2,N62769,Wise choices for stylish updating of old homes,NT1
2,2,SUMM3,N36186,Verlander may be reconsidering his stance on M...,NT1
3,3,SUMM4,N101669,Infamous o.j. Simpson launching official Twitt...,NT1
4,4,SUMM5,N19241,15 year old cori gauff beats Venus Williams at...,NT1
...,...,...,...,...,...
20595,20595,SUMM20596,N109539,Maricopa County first death was recorded from ...,NT103
20596,20596,SUMM20597,N41715,The 2019 Infiniti QX50 shows the future of its...,NT103
20597,20597,SUMM20598,N12997,Cronos Group may be the deal of a lifetime,NT103
20598,20598,SUMM20599,N17788,How health insurance companies make psychic he...,NT103


In [4]:
test_engage=pd.read_csv("w2p_test_engage_list.csv")
test_engage

,Unnamed: 0,EdgeID,Head,Relation,Tail,User
0,0,E1,NT1,skip,N11866,NT1
1,1,E2,N11866,skip,N81592,NT1
2,2,E3,N81592,click,N52295,NT1
3,3,E4,N52295,click,N100144,NT1
4,4,E5,N100144,skip,N17648,NT1
...,...,...,...,...,...,...
39665,39665,E39666,N90139,summ_gen,SUMM20448,NT103
39666,39666,E39667,SUMM20448,gen_summ,N79272,NT103
39667,39667,E39668,N79272,summ_gen,SUMM20449,NT103
39668,39668,E39669,SUMM20449,gen_summ,N99004,NT103


In [8]:
# Get max EdgeID number (strip 'E' and convert to int)
max_edge_num = test_engage['EdgeID'].str[1:].astype(int).max()
next_edge_num = max_edge_num + 1

new_edges = []

# Loop through each user with tqdm
for user, group in tqdm(test_sum.groupby("userid"), desc="Processing users"):
    # Already used summaries (first 50 in test_engage)
    used_summids = set(test_engage.loc[test_engage['User'] == user, 'Head']) | \
                   set(test_engage.loc[test_engage['User'] == user, 'Tail'])
    
    # Remaining summaries (exclude already used ones)
    remaining = group[~group['summid'].isin(used_summids)]
    
    for _, row in remaining.iterrows():
        summid = row['summid']   # e.g., SUMM101
        newsid = row['newsid']   # e.g., N12345

        # (summid → newsid) : gen_summ
        new_edges.append([f"E{next_edge_num}", summid, "gen_summ", newsid, user])
        next_edge_num += 1
        
        # (newsid → summid) : summ_gen
        new_edges.append([f"E{next_edge_num}", newsid, "summ_gen", summid, user])
        next_edge_num += 1

# Convert to DataFrame
new_edges_df = pd.DataFrame(new_edges, columns=["EdgeID", "Head", "Relation", "Tail", "User"])

# Append to original engage list
test_engage_updated = pd.concat([test_engage, new_edges_df], ignore_index=True)


Processing users: 100%|██████████| 103/103 [00:00<00:00, 144.63it/s]


In [9]:
test_engage_updated

,Unnamed: 0,EdgeID,Head,Relation,Tail,User
0,0.0,E1,NT1,skip,N11866,NT1
1,1.0,E2,N11866,skip,N81592,NT1
2,2.0,E3,N81592,click,N52295,NT1
3,3.0,E4,N52295,click,N100144,NT1
4,4.0,E5,N100144,skip,N17648,NT1
...,...,...,...,...,...,...
70571,NaN,E70572,N86284,summ_gen,SUMM19798,NT99
70572,NaN,E70573,SUMM19799,gen_summ,N100284,NT99
70573,NaN,E70574,N100284,summ_gen,SUMM19799,NT99
70574,NaN,E70575,SUMM19800,gen_summ,N57555,NT99


In [10]:
# Save updated CSV
test_engage_updated.to_csv("w2p_test_engage_list_updated.csv", index=False)

In [11]:
import pandas as pd
import pickle

# Load the test summaries
test_sum = pd.read_csv("test_summ.csv")

# Create mapping {summid: rewritten_title}
sid2sum = dict(zip(test_sum["summid"], test_sum["rewritten_title"]))

# Save to pickle
with open("sid2sum_test.pkl", "wb") as f:
    pickle.dump(sid2sum, f)
